# Notebook For Experimentation With ChatGPT Translation Features

In [67]:
import os
import threading
import tiktoken as tt
from openai import OpenAI
from pathlib import Path
from dotenv import load_dotenv

load_dotenv("./.env")  # take environment variables from .env.

True

In [79]:
model_name="gpt-3.5-turbo-0125"

In [11]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model_name,
)
print(chat_completion.choices[0].message.content)

This is a test.


Upload File

In [12]:
files_dir = "./files/"
trans_files_dir = "./translated_files/"

In [15]:
# Upload file to assistant
file = client.files.create(
    file=Path(f"{files_dir}mobile_home_utah_law.pdf"),
    purpose="assistants",
)

Retrieve Assistant

In [18]:
# Retrieve e2a assistant to translate document
assistant = client.beta.assistants.update(
  instructions="You are an English to Spanish translation bot. Draft me a similar document that translates the English text to Spanish.",
  model=model_name,
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

In [25]:
# Create a thread
my_thread = client.beta.threads.create()

In [26]:
# Get translation
message = client.beta.threads.messages.create(
  thread_id=my_thread.id,
  role="user",
  content="Translate this document to Spanish.",
  file_ids=[file.id]
)

In [27]:
# Run Assistant
my_run = client.beta.threads.runs.create(
  thread_id=my_thread.id,
  assistant_id=assistant.id,
  instructions="Please use the same formatting as in the original document."
)

In [41]:
print(my_run.status)

queued


In [42]:
all_messages = None
while my_run.status in ["queued", "in_progress"]:
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=my_thread.id,
        run_id=my_run.id
    )
    if keep_retrieving_run.status == "completed":
        print("\n")

        # Retrieve the Messages added by the Assistant to the Thread
        all_messages = client.beta.threads.messages.list(
            thread_id=my_thread.id
        )

        print("------------------------------------------------------------ \n")

        print(f"User: {message.content[0].text.value}")
        print(f"Assistant: {all_messages.data[0].content[0].text.value}")

        break
    elif keep_retrieving_run.status == "queued" or keep_retrieving_run.status == "in_progress":
        pass
    else:
        print(f"Run status: {keep_retrieving_run.status}")
        break




------------------------------------------------------------ 

User: Translate this document to Spanish.
Assistant: Capítulo 16
Ley de Residencia en Parques de Casas Móviles de Utah

57-16-1 Título corto.
Esta ley será conocida y podrá ser citada como la "Ley de Residencia en Parques de Casas Móviles".

Promulgada por la Sesión General de 1981, Capítulo 178

57-16-2 Propósito del capítulo.
El derecho fundamental a poseer y proteger la tierra y establecer condiciones para su uso por otros exige que al propietario de un parque de casas móviles se le otorguen remedios rápidos y adecuados contra aquellos que abusan de los términos de un arrendamiento. El alto costo de mover casas móviles, los requisitos de los parques de casas móviles relacionados con su instalación y el costo de ajardinar y preparar los terrenos requieren que los propietarios de casas móviles ocupadas dentro de parques de casas móviles estén protegidos contra desalojos reales o constructivos. El propósito de este capítu

In [78]:
m_l = client.beta.threads.messages.list(
            thread_id=my_thread.id,
            order="asc"
        )
# print(m_l.data[4].content[0].text.value)

print(keep_retrieving_run.usage)
enc = tt.encoding_for_model(model_name)

print(len(enc.encode(m_l.data[5].content[0].text.value)))

Usage(completion_tokens=2978, prompt_tokens=37817, total_tokens=40795)
None
2697
